In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

In [3]:
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [4]:
stk_object.dt = 2.5
AER_Data = stk_object.Get_Access_DP(stk_object.targets,
                                    stk_object.satellites,
                                    "AER Data/Default",
                                    ['Access Number','Time','Azimuth','Elevation'])
Cross_Track = stk_object.Get_Access_DP(stk_object.satellites,
                                       stk_object.targets,
                                       "Sat Angles Data",
                                       ["Cross Track"])

- Computing_AER Data/Default [==========] 150/150 [100%] in 20.4s (7.36/s)      
- Computing_Sat Angles Data [==========] 150/150 [100%] in 6.8s (22.28/s)       


In [5]:
total_dict = {key:[] for key in AER_Data[0][0]}
total_dict['Satellite'] = []
total_dict['Target'] = []
total_dict['Cross Track'] = []

tar_bin_count = np.zeros([len(stk_object.targets),36,9])
for tar_num,tar in enumerate(stk_object.targets):
    data_holder = AER_Data[tar_num].copy()
    for sat_num,df in enumerate(data_holder):
        df['Elevation'] = np.abs(df['Elevation'])
        df['Target'] = tar_num*np.ones_like(df[list(df.keys())[0]]).astype(int)
        df['Satellite'] = sat_num*np.ones_like(df[list(df.keys())[0]]).astype(int)
        df['Cross Track'] = Cross_Track[sat_num][tar_num]['Cross Track']
        for key in total_dict:
            total_dict[key].extend(df[key])

bins = []
for idx in range(len(total_dict['Target'])):
    az = total_dict['Azimuth'][idx]
    el = total_dict['Elevation'][idx]
    bins.append(int(az//10*9+el//10))

total_dict["Bin Number"] = bins
total_dict = pd.DataFrame(total_dict).sort_values(by="Time")

In [6]:
hash_map = {}
for tar_num in range(len(stk_object.targets)):
    hash_map[tar_num] = {}
    tar_window = total_dict['Target'].values==tar_num
    for bin_num in range(324):
        bin_window = total_dict['Bin Number'].values==bin_num
        hash_map[tar_num][bin_num] = {"Time":total_dict['Time'].values[bin_window&tar_window],
                                               "Satellite":total_dict['Satellite'].values[bin_window&tar_window].astype(int),
                                               "Cross Track":total_dict['Cross Track'].values[bin_window&tar_window]}
percentages = [len(np.unique(total_dict[total_dict['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
for tar_num,percentage in enumerate(percentages):
    print(f"Target_{tar_num} : {percentage:.2f}%")
print(f"Total : {sum(percentages)/len(percentages):.2f}%")

Target_0 : 88.58%
Target_1 : 95.06%
Target_2 : 85.49%
Target_3 : 88.58%
Target_4 : 91.05%
Target_5 : 90.43%
Target_6 : 79.01%
Target_7 : 99.69%
Target_8 : 91.36%
Target_9 : 78.70%
Target_10 : 81.79%
Target_11 : 84.57%
Target_12 : 83.64%
Target_13 : 95.06%
Target_14 : 84.57%
Total : 87.84%


In [7]:
tar_chosen = []
for tar_num in range(len(stk_object.targets)):
    tar_chosen.append([])
    for bin_num in range(324):
        if len(hash_map[tar_num][bin_num]['Time']) > 0:
            tar_chosen[-1].append([hash_map[tar_num][bin_num][key][0] for key in ['Time','Satellite','Cross Track']])
percentages = [len(chosen)/324*100 for chosen in tar_chosen]
for tar_num,percentage in enumerate(percentages):
    print(f"Target_{tar_num} : {percentage:.2f}%")
print(f"Total : {sum(percentages)/len(percentages):.2f}%")

Target_0 : 88.58%
Target_1 : 95.06%
Target_2 : 85.49%
Target_3 : 88.58%
Target_4 : 91.05%
Target_5 : 90.43%
Target_6 : 79.01%
Target_7 : 99.69%
Target_8 : 91.36%
Target_9 : 78.70%
Target_10 : 81.79%
Target_11 : 84.57%
Target_12 : 83.64%
Target_13 : 95.06%
Target_14 : 84.57%
Total : 87.84%


In [8]:
stk_object.dt = 2.5
stk_object.Interpolate = False
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")
stk_object.Compute_AzEl()
percentages = [100*np.count_nonzero(stk_object.target_bins[idx])/324 for idx in range(len(stk_object.targets))]
times = [stk_object.target_times[idx]/86400 for idx in range(len(stk_object.targets))]
for tar_num,percentage in enumerate(percentages):
    print(f"Target_{tar_num} : {percentage:.2f}%")
print(f"Total : {sum(percentages)/len(percentages):.2f}%")

- Computing_AzEl [==========] 150/150 [100%] in 18.5s (8.11/s)                  ▄▆█ 57/150 [38%] in 7s (~12s, 8.0/s) 
Target_0 : 88.58%
Target_1 : 95.06%
Target_2 : 85.49%
Target_3 : 88.58%
Target_4 : 91.05%
Target_5 : 90.43%
Target_6 : 79.01%
Target_7 : 99.69%
Target_8 : 91.36%
Target_9 : 78.70%
Target_10 : 81.79%
Target_11 : 84.57%
Target_12 : 83.64%
Target_13 : 95.06%
Target_14 : 84.57%
Total : 87.84%


In [1]:
def check_manueverability(theta1,time1,theta2,time2):
    dtheta = abs(theta2-theta1)
    dtime = time2-time1
    if dtime == 0:
        return False,0
    else:
        return dtheta/dtime<0.1,dtime

In [ ]:
Nodes = []
Edges = []
captured_bins = [[] for _ in range(len(stk_object.targets))]
for sat_num in range(len(stk_object.satellites))[:1]:
    sat_df = total_dict[total_dict['Satellite'].values==sat_num]
    for i in range(len(sat_df))[:96]:
        Nodes.append(i)
        Edges.append([])
        bin_window = sat_df['Bin Number'].values == sat_df['Bin Number'].values[i]
        target_window = sat_df['Target'].values == sat_df['Target'].values[i]
        time_window = (sat_df['Time'].values < sat_df['Time'].values[i]+200)&(sat_df['Time'].values > sat_df['Time'].values[i])
        new_df = sat_df[time_window&~(bin_window&target_window)]
        for j in range(len(new_df)):
            feasible,transition_time = check_manueverability(sat_df['Cross Track'].values[i],
                                                            sat_df['Time'].values[i],
                                                            new_df['Cross Track'].values[j],
                                                            new_df['Time'].values[j])
            if feasible and j!=i:
                Edges[-1].append(j+i+1)
        # if len(Edges[-1])==0:
        #     Edges[-1].append(i+1)
            
network = {key:value for key,value in zip(Nodes,Edges)}

sum([len(case) for case in Edges])

In [ ]:
node_x = []
node_y = []

edge_x = []
edge_y = []

for idx,node in enumerate(network):
    node_x.append(sat_df['Time'].values[idx])
    node_y.append(sat_df['Target'].values[idx])
    for edge in network[node]:
        edge_x.append(sat_df['Time'].values[idx])
        edge_x.append(sat_df['Time'].values[edge])
        edge_y.append(sat_df['Target'].values[idx])
        edge_y.append(sat_df['Target'].values[edge])

node_trace = go.Scatter(
    x=node_x, y=node_y,
    line=dict(width=0.5, color='blue'),
    hovertext=Nodes,
    mode='markers')

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='grey'),
    hoverinfo='none',
    mode='lines')

fig = go.Figure()
fig.add_trace(edge_trace)
fig.add_trace(node_trace)
fig.show()